# 캐치 크롤링
- 크롤링할 컬럼: 기업명, 기업형태, 매출액, 사원수, 업종분류, 설립연도
- 절차
1. url로 기업명 입력
2. 기업정보 사이트 접속
3. 내용 크롤링
4. concat
5. 다음 url 입력

위 절차를 반복합니다.

In [ ]:
import pandas as pd
# 정적크롤링
from bs4 import BeautifulSoup
import requests
import re
# 동적크롤링
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

In [ ]:
# 파일 불러오기: 불러올 파일을 현재 폴더에 넣어두세요.
df = pd.read_csv('./programmers.csv')  # 파일명만 변경하여 그대로 돌리시면 됩니다.
df_corp_name = df['기업명']  # 기업명 불러오기

In [ ]:
# 임시 프레임
catch_res = pd.DataFrame(columns=['기업명','기업형태','매출액','사원수','업종분류','설립연도'])

In [ ]:
# 크롤링 프로그램
for name in df_corp_name:
    try:
        url = f'https://www.catch.co.kr/Search/SearchDetail?Menu=2&Keyword={name}'

        driver= webdriver.Chrome()
        driver.get(url)

        driver.find_element(By.XPATH, '//*[@id="leftTop"]/ul/li/div[2]/p[1]/a').click()
        time.sleep(0.2)

        # 기업명
        corp_name = name
        # 기업형태
        corp_type = driver.find_element(By.XPATH, '//*[@id="Contents"]/div[2]/div[1]/table/tbody/tr[2]/td[1]').text
        # 매출액
        sales_money = driver.find_element(By.XPATH, '//*[@id="Contents"]/div[2]/div[1]/table/tbody/tr[3]/td[1]').text
        # 사원수
        employee = driver.find_element(By.XPATH, '//*[@id="Contents"]/div[2]/div[1]/table/tbody/tr[3]/td[2]').text
        # 업종분류
        try:
            sector = driver.find_element(By.XPATH, '//*[@id="Contents"]/div[2]/div[1]/table/tbody/tr[5]/td[1]').text
        except:
            sector = driver.find_element(By.XPATH, '//*[@id="Contents"]/div[2]/div[1]/table/tbody/tr[4]/td[1]').text
        # 설립연도
        corp_year = driver.find_element(By.XPATH, '//*[@id="Contents"]/div[2]/div[1]/table/tbody/tr[1]/td[2]').text

        catch_temp = pd.DataFrame([corp_name, corp_type, sales_money, employee, sector, corp_year],
                                   index=['기업명','기업형태','매출액','사원수','업종분류','설립연도']).T
        catch_res = pd.concat([catch_res, catch_temp])

        time.sleep(0.2)
    except:
        pass
    
    driver.quit()

In [ ]:
catch_res.to_csv('./catch_result2.csv')  # 파일명만 변경하여 저장하시면 됩니다.

# 코드 수정

In [1]:
import pandas as pd
# 정적크롤링
from bs4 import BeautifulSoup
import requests
import re
# 동적크롤링
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

In [2]:
# 파일 불러오기: 불러올 파일을 현재 폴더에 넣어두세요.
df = pd.read_csv('./programmers.csv')  # 파일명만 변경하여 그대로 돌리시면 됩니다.
df_corp_name = df['기업명']  # 기업명 불러오기

In [3]:
# 임시 프레임
catch_res = pd.DataFrame(columns=['기업명','기업형태','매출액','사원수','업종분류','설립연도'])

In [4]:
# 크롤링 프로그램
for name in df_corp_name:
    url = f'https://www.catch.co.kr/Search/SearchDetail?Menu=2&Keyword={name}'  # 기업명으로 URL불러오기

    driver= webdriver.Chrome()
    driver.get(url)
    
    try:
        driver.find_element(By.XPATH, '//*[@id="leftTop"]/ul/li/div[2]/p[1]/a').click()
        time.sleep(0.2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 기업명
        corp_name = name
        corp_year = soup.select('td')[1].text  # 설립연도
        corp_type = soup.select('td')[2].text  # 기업 형태
        if len(soup.select('td')) == 10:
            sales_money = soup.select('td')[4].text  # 매출액
            employee = soup.select('td')[8].text  # 사원수
            sector = soup.select('td')[9].text  # 업종분류
        elif len(soup.select('td')) == 8:
            sales_money = soup.select('td')[4].text  # 매출액
            employee = soup.select('td')[5].text  # 사원수
            sector = soup.select('td')[6].text  # 업종분류        
        else:
            employee = soup.select('td')[4].text  # 사원수
            sector = soup.select('td')[5].text  # 업종분류

        catch_temp = pd.DataFrame([corp_name, corp_type, sales_money, employee, sector, corp_year],
                                   index=['기업명','기업형태','매출액','사원수','업종분류','설립연도']).T
        catch_res = pd.concat([catch_res, catch_temp])

        time.sleep(0.2)
    except:
        pass
    
driver.quit()

In [28]:
catch_res.to_csv('./catch_result.csv')  # 파일명만 변경하여 저장하시면 됩니다.